In [ ]:
# !pip install torchvision scikit-image pillow==4.1.1

In [1]:
!pip install https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip

     - 17 kB 4.5 MB/s
     \ 131 kB 5.3 MB/s
     |████████████████████████████████| 3.3 MB 13.1 MB/s 
     |████████████████████████████████| 60.3 MB 1.5 MB/s 
     |████████████████████████████████| 3.1 MB 41.6 MB/s 
  Created wheel for hpacellseg: filename=hpacellseg-0.1.8-py3-none-any.whl size=14962 sha256=c1b98113967f3a445a9ec9cceee3b8479a9efe837cc036320c23ac0aee7e0090
  Stored in directory: /tmp/pip-ephem-wheel-cache-9o7gtiry/wheels/cd/d8/de/04ad08802d62537f8dffc89b6a7ce0a53c3d29ea6eae522ab1
  Created wheel for pytorch-zoo: filename=pytorch_zoo-0.0.0-py3-none-any.whl size=30138 sha256=3cc4dd74d69f320f36e23579cb0a01d4a2ef3c8f796337418bc3dc194a5c2afc
  Stored in directory: /tmp/pip-ephem-wheel-cache-9o7gtiry/wheels/58/0e/c7/567928a140c7cb2533c59bdc81fded8fe720ea8caad1659d8c
Successfully built hpacellseg pytorch-zoo
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting unin

In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
# For utilities
import os, shutil, time


In [2]:
use_gpu = torch.cuda.is_available()

In [3]:
!wget http://data.csail.mit.edu/places/places205/testSetPlaces205_resize.tar.gz
!tar -xzf testSetPlaces205_resize.tar.gz

--2021-12-07 12:11:22--  http://data.csail.mit.edu/places/places205/testSetPlaces205_resize.tar.gz
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.129.40
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.129.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2341250899 (2.2G) [application/octet-stream]
Saving to: ‘testSetPlaces205_resize.tar.gz.1’

testSetPlaces205_re 100%[===================>]   2.18G  3.07MB/s    in 18m 5s  

2021-12-07 12:29:27 (2.06 MB/s) - ‘testSetPlaces205_resize.tar.gz.1’ saved [2341250899/2341250899]


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [4]:
!tar -xzf /content/testSetPlaces205_resize.tar.gz.1

In [5]:
import os
import shutil
os.makedirs('images/train/class/', exist_ok=True) # 40,000 images
os.makedirs('images/val/class/', exist_ok=True)   #  1,000 images
for i, file in enumerate(os.listdir('testSet_resize')):
  if i < 1000: # first 1000 will be val
    shutil.move('testSet_resize/' + file, 'images/val/class/' + file)
  else: # others will be val
    shutil.move('testSet_resize/' + file, 'images/train/class/' + file)


In [6]:
len(os.listdir("/content/images/train/class"))

40000

In [41]:
class ColorizationNet(nn.Module):
  def __init__(self, input_size=128):
    super(ColorizationNet, self).__init__()

    MIDLEVEL_FEATURE_SIZE = 128
    resnet=models.resnet18(pretrained=True)
    resnet.conv1.weight=nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1))
    
    self.midlevel_resnet =nn.Sequential(*list(resnet.children())[0:6])

    self.upsample = nn.Sequential(     
      nn.Conv2d(MIDLEVEL_FEATURE_SIZE, 128, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1),
      nn.Upsample(scale_factor=2)
    )

  def forward(self, input):

    # Pass input through ResNet-gray to extract features
    midlevel_features = self.midlevel_resnet(input)

    # Upsample to get colors
    output = self.upsample(midlevel_features)
    return output

    

In [45]:
model=ColorizationNet()

In [46]:
model(torch.rand(1,1,224,224)).shape

torch.Size([1, 2, 224, 224])

In [10]:
class GrayscaleImageFolder(datasets.ImageFolder):
  '''Custom images folder, which converts images to grayscale before loading'''
  def __getitem__(self, index):
    path, target = self.imgs[index]
    img = self.loader(path)
    if self.transform is not None:
      img_original = self.transform(img)
      img_original = np.asarray(img_original)
      img_lab = rgb2lab(img_original)
      img_lab = (img_lab + 128) / 255
      img_ab = img_lab[:, :, 1:3]
      img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
      img_original = rgb2gray(img_original)
      img_original = torch.from_numpy(img_original).unsqueeze(0).float()
    if self.target_transform is not None:
      target = self.target_transform(target)
    return img_original, img_ab, target

In [47]:
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip()])
train_imagefolder = GrayscaleImageFolder('images/train', train_transforms)
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=64, shuffle=True)


In [48]:
len(train_loader)

625

In [13]:
# class AverageMeter(object):
#   '''A handy class from the PyTorch ImageNet tutorial''' 
#   def __init__(self):
#     self.reset()
#   def reset(self):
#     self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
#   def update(self, val, n=1):
#     self.val = val
#     self.sum += val * n
#     self.count += n
#     self.avg = self.sum / self.count



In [14]:
#out.de.shape

In [15]:
#input.shape()

In [16]:
#plt.imshow(show_output(input.squeeze(0),out.squeeze(0).cpu()))

In [42]:
def train(train_loader, model, criterion, optimizer, epoch):
  print('Starting training epoch {}'.format(epoch))
  model.train()

  for i, (input_gray, input_ab, target) in enumerate(train_loader):
    
    if use_gpu: input_gray, input_ab, target = input_gray.cuda(), input_ab.cuda(), target.cuda()

    output_ab = model(input_gray) 
    loss = criterion(output_ab, input_ab) 
    # losses.update(loss.item(), input_gray.size(0))

    # Compute gradient and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print model accuracy -- in the code below, val refers to value, not validation
    if i % 25 == 0:
      print('Epoch: [{0}][{1}/{2}]\t'
            'Loss {loss:.4f} ({loss:.4f})\t'.format(
              epoch, i, len(train_loader), loss=loss)) 

  print('Finished training epoch {}'.format(epoch))


In [18]:
#from IPython.display import Image, display
#display(Image(filename='/content/images/val/class/000d91ee9907dc11ecb76a0d4dd54cec.jpg'))

In [43]:
import cv2
from google.colab.patches import cv2_imshow
im=cv2.imread("/content/b&w.jpg")
cv2_imshow(im)

lab=rgb2lab(im)
cv2_imshow(lab)


AttributeError: ignored

In [ ]:
#lab.shape

In [ ]:
#im.shape

In [21]:

if use_gpu: 
  criterion = criterion.cuda()
  model = model.cuda()


NameError: ignored

In [ ]:

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=0.0)
model=model.cuda()

for epoch in range(10):
  # Train for one epoch, then validate
  train(train_loader, model, criterion, optimizer, epoch)
  # torch.save(model.state_dict(), 'checkpoints/model-epoch-{}-losses-{:.3f}.pth'.format(epoch+1,loss))
 

Starting training epoch 0
Epoch: [0][0/625]	Loss 0.5006 (0.5006)	
Epoch: [0][25/625]	Loss 0.4979 (0.4979)	
Epoch: [0][50/625]	Loss 0.4945 (0.4945)	
Epoch: [0][75/625]	Loss 0.4890 (0.4890)	
Epoch: [0][100/625]	Loss 0.4737 (0.4737)	
Epoch: [0][125/625]	Loss 0.4724 (0.4724)	
Epoch: [0][150/625]	Loss 0.4600 (0.4600)	
Epoch: [0][175/625]	Loss 0.4621 (0.4621)	
Epoch: [0][200/625]	Loss 0.4606 (0.4606)	
Epoch: [0][225/625]	Loss 0.4424 (0.4424)	
Epoch: [0][250/625]	Loss 0.4442 (0.4442)	
Epoch: [0][275/625]	Loss 0.4345 (0.4345)	
Epoch: [0][300/625]	Loss 0.4234 (0.4234)	
Epoch: [0][325/625]	Loss 0.4316 (0.4316)	
Epoch: [0][350/625]	Loss 0.4129 (0.4129)	
Epoch: [0][375/625]	Loss 0.4036 (0.4036)	
Epoch: [0][400/625]	Loss 0.3988 (0.3988)	
Epoch: [0][425/625]	Loss 0.3908 (0.3908)	
Epoch: [0][450/625]	Loss 0.3854 (0.3854)	
Epoch: [0][475/625]	Loss 0.3796 (0.3796)	
Epoch: [0][500/625]	Loss 0.3623 (0.3623)	
Epoch: [0][525/625]	Loss 0.3555 (0.3555)	
Epoch: [0][550/625]	Loss 0.3391 (0.3391)	
Epoch: [0][57

In [33]:
torch.save(model.state_dict(), 'model-final.pth')


In [34]:
model.load_state_dict(torch.load("/content/model-final.pth"))

<All keys matched successfully>

In [24]:
torch.Tensor(rgb2gray(im)).unsqueeze(0).unsqueeze(0)


AttributeError: ignored

In [ ]:
def colorize(img_path, print_img=True):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    grayscale_input = torch.Tensor(rgb2gray(img))
    ab_input = model(grayscale_input.unsqueeze(0).unsqueeze(0)).squeeze(0)
    predicted = show_output(grayscale_input.unsqueeze(0), ab_input)
    if print_img:
        plt.imshow(predicted)
    return predicted
def show_output(grayscale_input, ab_input):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    color_image = torch.cat((grayscale_input, ab_input), 0).detach().numpy()  # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    # plt.imshow(grayscale_input)
    # plt.imshow(color_image)
    return color_image

In [38]:
model=torch.load("/content/model-final.pth")
#model=ColorizationNet().cuda()
#model.load_state_dict(torch.load("/content/model-final.pth"))

In [ ]:
out = colorize("/content/b&w.jpg")
print(out)

RuntimeError: ignored

In [ ]:
input=torch.Tensor(rgb2gray(im)).unsqueeze(0).unsqueeze(0).cuda()
out=model(input)
#out=(torch.cat([out,input],dim=1).squeeze(0)).permute([1,2,0])
# plt.imshow(out.detach().numpy())

In [ ]:
out=(torch.cat([out,input],dim=1).squeeze(0)).permute([1,2,0])


RuntimeError: ignored

In [ ]:
print(input.shape)
out.shape

torch.Size([1, 1, 2975, 2082])


torch.Size([1, 2, 2976, 2088])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# shutil.copy("/content/model-epoch-2.pth","/content/drive/MyDrive/Datasets/Models/Image Colorizer")

In [ ]:
def show_output(grayscale_input, ab_input):
  '''Show/save rgb image from grayscale and ab channels
     Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
  color_image = torch.cat((grayscale_input, ab_input), 0).detach().numpy() # combine channels
  color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
  color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
  color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
  color_image = lab2rgb(color_image.astype(np.float64))
  grayscale_input = grayscale_input.squeeze().numpy()
  # plt.imshow(grayscale_input)
  # plt.imshow(color_image)
  return color_image


In [ ]:
model=model.cpu()


In [ ]:
def colorize(img_path,print_img=True):
    grayscale_input= torch.Tensor(rgb2gray(cv2.imread(img_path)))
    ab_input=model(grayscale_input.unsqueeze(0).unsqueeze(0)).squeeze(0)
    predicted=show_output(grayscale_input.unsqueeze(0), ab_input)
    plt.imshow(predicted)
    return predicted

In [ ]:
colorize("/content/images/val/class/00bd27e623e062785c317ef58ad6e7c0.jpg")